In [ ]:
import nbimporter
import json
import requests
import time
import datetime
import urllib

# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json

# create trading instance
trading = betfairlightweight.APIClient("username", "password", app_key="appKey")

# login
trading.login()

## Get Next Race

In [11]:
#Call APING
#Ensure connection to API
def callAping(jsonrpc_req):
    try:
        req = urllib.request.Request(url, jsonrpc_req.encode('utf-8'), headers)
        response = urllib.request.urlopen(req)
        jsonResponse = response.read()
        return jsonResponse.decode('utf-8')
    except urllib.error.URLError as e:
        print (e.reason) 
        print ('Oops no service available at ' + str(url))
        exit()
    except urllib.error.HTTPError:
        print ('Oops not a valid operation from the service ' + str(url))
        exit()


In [12]:
"""
calling getEventTypes operation
"""

def getEventTypes():
    event_type_req = '{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listEventTypes", "params": {"filter":{ }}, "id": 1}'
    print ('Calling listEventTypes to get event Type ID')
    eventTypesResponse = callAping(event_type_req)
    eventTypeLoads = json.loads(eventTypesResponse)
    """
print(eventTypeLoads)
"""

    try:
        eventTypeResults = eventTypeLoads['result']
        return eventTypeResults
    except:
        print ('Exception from API-NG' + str(eventTypeLoads['error']))
        exit()


In [13]:
"""
Extraction eventypeId for eventTypeName from evetypeResults
"""
def getEventTypeIDForEventTypeName(eventTypesResult, requestedEventTypeName):
    if(eventTypesResult is not None):
        for event in eventTypesResult:
            eventTypeName = event['eventType']['name']
            if( eventTypeName == requestedEventTypeName):
                return event['eventType']['id']
    else:
        print ('Oops there is an issue with the input')
        exit()


In [14]:
"""
Calling marketCatalouge to get marketDetails
"""

def getMarketCatalogueForNextGBWin(eventTypeID):
    if (eventTypeID is not None):
        print ('Calling listMarketCatalouge Operation to get MarketID and selectionId')
        now = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ')
        market_catalogue_req = '{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listMarketCatalogue", "params": {"filter":{"eventTypeIds":["7"],"marketTypeCodes":["WIN"],'\
                                                                                                                                                             '"marketStartTime":{"from":"' + now + '"}},"sort":"FIRST_TO_START","maxResults":"1","marketProjection":["RUNNER_METADATA"]}, "id": 1}'
        """
print(market_catalogue_req)
"""
        market_catalogue_response = callAping(market_catalogue_req)
        """
print(market_catalogue_response)
"""
        market_catalouge_loads = json.loads(market_catalogue_response)
        
        try:
            market_catalouge_results = market_catalouge_loads['result']
            return market_catalouge_results
        except:
            print ('Exception from API-NG' + str(market_catalouge_results['error']))
            exit()


def getMarketId(marketCatalogueResult):
    if( marketCatalogueResult is not None):
        for market in marketCatalogueResult:
            return market['marketId']


def getSelectionId(marketCatalogueResult):
    if(marketCatalogueResult is not None):
        for market in marketCatalogueResult:
            return market['runners'][0]['selectionId']


def getMarketBookBestOffers(marketId):
    print ('Calling listMarketBook to read prices for the Market with ID :' + marketId)
    market_book_req = '{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listMarketBook", "params": {"marketIds":["' + marketId + '"],"priceProjection":{"priceData":["EX_BEST_OFFERS"]}}, "id": 1}'
    """
print(market_book_req)
"""
    market_book_response = callAping(market_book_req)
    """
print(market_book_response)
"""
    market_book_loads = json.loads(market_book_response)
    try:
        market_book_result = market_book_loads['result']
        return market_book_result
    except:
        print ('Exception from API-NG' + str(market_book_result['error']))
        exit()


def printPriceInfo(market_book_result):
    if(market_book_result is not None):
        print ('Please find Best three available prices for the runners')
        for marketBook in market_book_result:
            runners = marketBook['runners']
            for runner in runners:
                print ('Selection id is ' + str(runner['selectionId']))
                if (runner['status'] == 'ACTIVE'):
                    print ('Available to back price :' + str(runner['ex']['availableToBack']))
                    print ('Available to lay price :' + str(runner['ex']['availableToLay']))
                else:
                    print ('This runner is not active')



In [15]:
def placeShittyBet(marketId, selectionId):
    
     if( marketId is not None and selectionId is not None):
            
        print ('Calling placeOrder for marketId :' + marketId + ' with selection id :' + str(selectionId))
        place_order_Req = '{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/placeOrders", "params": {"marketId":"' + str(marketId) + '","instructions":'\
                                                                                                                              '[{"selectionId":"' + str(
            selectionId) + '","handicap":"0","side":"BACK","orderType":"LIMIT","limitOrder":{{"size":"19.09","price":"3.40"},"persistenceType":"LAPSE"}}]}, "id": 1}'
        """
        3.4, 'size': 19.09
print(place_order_Req)
"""
        place_order_Response = callAping(place_order_Req)
        place_order_load = json.loads(place_order_Response)
        try:
            place_order_result = place_order_load['result']
            print ('Place order status is ' + place_order_result['status'])
            """
print('Place order error status is ' + place_order_result['errorCode'])
"""
            print ('Reason for Place order failure is ' + place_order_result['instructionReports'][0]['errorCode'])
        except:
            print ('Exception from API-NG' + str(place_order_result['error']))
        """
print(place_order_Response)
"""


In [16]:
url = "https://api.betfair.com/exchange/betting/json-rpc/v1"

"""
headers = { 'X-Application' : 'xxxxxx', 'X-Authentication' : 'xxxxx' ,'content-type' : 'application/json' }
"""
appKey = ''
##SSID
sessionToken = ''
headers = {'X-Application': appKey, 'X-Authentication': sessionToken, 'content-type': 'application/json'}

eventTypesResult = getEventTypes()
horseRacingEventTypeID = getEventTypeIDForEventTypeName(eventTypesResult, 'Horse Racing')

print ('Eventype Id for Horse Racing is :' + str(horseRacingEventTypeID))

marketCatalogueResult = getMarketCatalogueForNextGBWin(horseRacingEventTypeID)
marketid = getMarketId(marketCatalogueResult)
runnerId = getSelectionId(marketCatalogueResult)



print(json.dumps(marketCatalogueResult, indent=4))
#print(runnerId)

market_book_result = getMarketBookBestOffers(marketid)
print(market_book_result)

printPriceInfo(market_book_result)

placeShittyBet(marketid, runnerId)

Calling listEventTypes to get event Type ID
Eventype Id for Horse Racing is :7
Calling listMarketCatalouge Operation to get MarketID and selectionId
[
    {
        "marketId": "1.204621419",
        "marketName": "R3 5f Mdn",
        "totalMatched": 0.0,
        "runners": [
            {
                "selectionId": 49159572,
                "runnerName": "Decorated Lady",
                "handicap": 0.0,
                "sortPriority": 1,
                "metadata": {
                    "SIRE_NAME": "Hard Aces",
                    "CLOTH_NUMBER_ALPHA": "1",
                    "OFFICIAL_RATING": null,
                    "COLOURS_DESCRIPTION": "Green, White, Red Stripes Body & Sleeves w/ DG",
                    "COLOURS_FILENAME": "saddle_1.gif",
                    "FORECASTPRICE_DENOMINATOR": "1",
                    "DAMSIRE_NAME": "Songandaprayer",
                    "WEIGHT_VALUE": "120",
                    "SEX_TYPE": "f",
                    "DAYS_SINCE_LAST_RUN": "0",

TypeError: the JSON object must be str, bytes or bytearray, not NoneType